In [1]:
import tensorflow as tf

# 这里声明的变量名称和已经保存的模型中名称不同
v1 = tf.Variable(tf.constant(1.0, shape=[1]), name='other-v1')
v2 = tf.Variable(tf.constant(2.0, shape=[1]), name='other-v2')
result = v1 + v2
'''
如果使用tf.train.Saver()来加载模型会报变量找不到的错误。
tensorflow.python.framework.errors_impl.NotFoundError: Key other-v1 not found in checkpoint
'''
# saver = tf.train.Saver()

# 声明tf.train.Saver类用于保存模型
'''
使用一个字典(dictionary)来重命名变量就可以加载原来的模型了。这个字典制定了原来名称为
v1的变量现在加载到变量v1中（名称为other-v1）v2 同理
'''
saver = tf.train.Saver({"v1": v1, "v2": v2})

E:\Anaconda3\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
'''
参数 初始化
'''
with tf.Session() as sess:
    # 加载已经保存的模型，并通过已经保存的模型中变量的值来计算加法
    saver.restore(sess, "Saved_model/model.ckpt")
    print(sess.run(result))

INFO:tensorflow:Restoring parameters from Saved_model/model.ckpt
[3.]


In [3]:
v = tf.Variable(0, dtype=tf.float32, name='v')
for var in tf.all_variables():
    print(var.name)

Instructions for updating:
Please use tf.global_variables instead.
other-v1:0
other-v2:0
v:0


In [4]:
ema = tf.train.ExponentialMovingAverage(0.99)
maintain_average_op = ema.apply(tf.all_variables())
for var in tf.all_variables():
    print(var.name)

other-v1:0
other-v2:0
v:0
other-v1/ExponentialMovingAverage:0
other-v2/ExponentialMovingAverage:0
v/ExponentialMovingAverage:0


In [5]:
saver = tf.train.Saver()
with tf.Session() as sess:
    init_op = tf.initialize_all_variables()
    sess.run(init_op)
    sess.run(tf.assign(v,10))
    sess.run(maintain_average_op)
    saver.save(sess, 'Saved_model/exponentialmoveave.ckpt')
    print(sess.run([v, ema.average(v)]))
    print(v.name)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
[10.0, 0.099999905]
v:0


In [6]:
v = tf.Variable(0, dtype=tf.float32, name='v')
saver = tf.train.Saver({'v/ExponentialMovingAverage':v})
with tf.Session() as sess:
    saver.restore(sess, 'Saved_model/exponentialmoveave.ckpt')
    print(sess.run(v))

INFO:tensorflow:Restoring parameters from Saved_model/exponentialmoveave.ckpt
0.099999905


In [7]:
v = tf.Variable(0, dtype=tf.float32, name='v')
ema = tf.train.ExponentialMovingAverage(0.99)
# 生成tf.train.Saver类所需要的变量重命名字典
print(ema.variables_to_restore())

{'v_1/ExponentialMovingAverage': <tf.Variable 'v_1:0' shape=() dtype=float32_ref>, 'v/ExponentialMovingAverage_1': <tf.Variable 'v:0' shape=() dtype=float32_ref>, 'v_2/ExponentialMovingAverage': <tf.Variable 'v_2:0' shape=() dtype=float32_ref>, 'other-v2/ExponentialMovingAverage_1': <tf.Variable 'other-v2:0' shape=(1,) dtype=float32_ref>, 'other-v1/ExponentialMovingAverage_1': <tf.Variable 'other-v1:0' shape=(1,) dtype=float32_ref>, 'other-v1/ExponentialMovingAverage': <tf.Variable 'other-v1/ExponentialMovingAverage:0' shape=(1,) dtype=float32_ref>, 'other-v2/ExponentialMovingAverage': <tf.Variable 'other-v2/ExponentialMovingAverage:0' shape=(1,) dtype=float32_ref>, 'v/ExponentialMovingAverage': <tf.Variable 'v/ExponentialMovingAverage:0' shape=() dtype=float32_ref>}


In [8]:
saver = tf.train.Saver(ema.variables_to_restore())
with tf.Session() as sess:
    saver.restore(sess, 'Saved_model/exponentialmoveave.ckpt')
    print(sess.run(v))

INFO:tensorflow:Restoring parameters from Saved_model/exponentialmoveave.ckpt


NotFoundError: Key other-v1/ExponentialMovingAverage_1 not found in checkpoint
	 [[Node: save_3/RestoreV2 = RestoreV2[dtypes=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save_3/Const_0_0, save_3/RestoreV2/tensor_names, save_3/RestoreV2/shape_and_slices)]]

Caused by op 'save_3/RestoreV2', defined at:
  File "E:\Anaconda3\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "E:\Anaconda3\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "E:\Anaconda3\tensorflow\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "E:\Anaconda3\tensorflow\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "E:\Anaconda3\tensorflow\lib\site-packages\ipykernel\kernelapp.py", line 486, in start
    self.io_loop.start()
  File "E:\Anaconda3\ten